# 使用Chroma进行嵌入搜索

本笔记本将带您完成一个简单的流程，下载一些数据，对其进行嵌入，然后使用一些向量数据库进行索引和搜索。这是客户经常需要的需求，他们希望在安全环境中存储和搜索我们的嵌入数据，以支持生产用例，如聊天机器人、主题建模等。

### 什么是向量数据库

向量数据库是一种用于存储、管理和搜索嵌入向量的数据库。近年来，使用嵌入来将非结构化数据（文本、音频、视频等）编码为向量，供机器学习模型使用的做法已经蓬勃发展，这是由于人工智能在解决涉及自然语言、图像识别和其他非结构化数据形式的用例时日益有效。向量数据库已经成为企业提供和扩展这些用例的有效解决方案。

### 为什么使用向量数据库

向量数据库使企业能够利用我们在这个存储库中分享的许多嵌入用例（例如问答、聊天机器人和推荐服务），并在安全、可扩展的环境中使用它们。许多客户在小规模上使用嵌入来解决问题，但性能和安全性阻碍了它们投入生产 - 我们认为向量数据库是解决这一问题的关键组成部分，在本指南中，我们将介绍嵌入文本数据的基础知识，将其存储在向量数据库中，并将其用于语义搜索。

### 演示流程
演示流程如下：
- **设置**：导入包并设置任何必需的变量
- **加载数据**：加载数据集并使用OpenAI嵌入进行嵌入
- **Chroma**：
    - *设置*：在这里，我们将设置Chroma的Python客户端。更多详情请查看[这里](https://docs.trychroma.com/usage-guide)
    - *索引数据*：我们将为__标题__和__内容__创建带有向量的集合
    - *搜索数据*：我们将运行一些搜索以确认其有效性

完成本笔记后，您应该对如何设置和使用向量数据库有基本的了解，并可以继续进行更复杂的用例，利用我们的嵌入。


## 设置

导入所需的库并设置我们想要使用的嵌入模型。


In [1]:
# 确保已安装 OpenAI 库。
%pip install openai

# 我们需要安装Chroma客户端。
%pip install chromadb

# 安装 wget 以拉取 zip 文件
%pip install wget

# 安装用于数据处理的numpy库
%pip install numpy


  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/67/78/7588a047e458cb8075a4089d721d7af5e143ff85a2388d4a28c530be0494/openai-0.27.8-py3-none-any.whl.metadata
  Obtaining dependency information for requests>=2.20 from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Obtaining dependency information for aiohttp from https://files.pythonhosted.org/packages/fa/9e/49002fde2a97d7df0e162e919c31cf13aa9f184537739743d1239edd0e67/aiohttp-3.8.5-cp310-cp310-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/ec/a7/96835706283d63fefbbbb4f119d52f195af00fc747e67cc54397c56312c8/charset_normalizer-3.2.0-cp310-cp310-macosx_11_0_arm64.whl.metadata
  Using cached c

In [3]:
import openai
import pandas as pd
import os
import wget
from ast import literal_eval

# Chroma's client library for Python
import chromadb

# I've set this to our new embeddings model, this can be changed to the embedding model of your choice
EMBEDDING_MODEL = "text-embedding-3-small"

# 忽略未关闭的SSL套接字警告——如果你遇到这些错误，可以选择此项。
import warnings

warnings.filterwarnings(action="ignore", message="unclosed", category=ResourceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 


## 加载数据

在本部分，我们将加载在本次会话之前准备好的嵌入数据。


In [3]:
embeddings_url = 'https://cdn.openai.com/API/examples/data/vector_database_wikipedia_articles_embedded.zip'

# 文件大小约为700MB，因此需要一些时间来完成。
wget.download(embeddings_url)


'vector_database_wikipedia_articles_embedded.zip'

In [4]:
import zipfile
with zipfile.ZipFile("vector_database_wikipedia_articles_embedded.zip","r") as zip_ref:
    zip_ref.extractall("../data")


In [7]:
article_df = pd.read_csv('../data/vector_database_wikipedia_articles_embedded.csv')


In [8]:
article_df.head()


,id,url,title,text,title_vector,content_vector,vector_id
0,1,https://simple.wikipedia.org/wiki/April,April,April is the fourth month of the year in the J...,"[0.001009464613161981, -0.020700545981526375, ...","[-0.011253940872848034, -0.013491976074874401,...",0
1,2,https://simple.wikipedia.org/wiki/August,August,August (Aug.) is the eighth month of the year ...,"[0.0009286514250561595, 0.000820168002974242, ...","[0.0003609954728744924, 0.007262262050062418, ...",1
2,6,https://simple.wikipedia.org/wiki/Art,Art,Art is a creative activity that expresses imag...,"[0.003393713850528002, 0.0061537534929811954, ...","[-0.004959689453244209, 0.015772193670272827, ...",2
3,8,https://simple.wikipedia.org/wiki/A,A,A or a is the first letter of the English alph...,"[0.0153952119871974, -0.013759135268628597, 0....","[0.024894846603274345, -0.022186409682035446, ...",3
4,9,https://simple.wikipedia.org/wiki/Air,Air,Air refers to the Earth's atmosphere. Air is a...,"[0.02224554680287838, -0.02044147066771984, -0...","[0.021524671465158463, 0.018522677943110466, -...",4


In [9]:
# 从字符串中读取向量并将其转换为列表
article_df['title_vector'] = article_df.title_vector.apply(literal_eval)
article_df['content_vector'] = article_df.content_vector.apply(literal_eval)

# 将 `vector_id` 设置为一个字符串
article_df['vector_id'] = article_df['vector_id'].apply(str)


In [11]:
article_df.info(show_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              25000 non-null  int64 
 1   url             25000 non-null  object
 2   title           25000 non-null  object
 3   text            25000 non-null  object
 4   title_vector    25000 non-null  object
 5   content_vector  25000 non-null  object
 6   vector_id       25000 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.3+ MB


# 色度

我们将在一个向量数据库中索引这些嵌入式文档并进行搜索。我们将首先看一下的选项是**Chroma**，这是一个易于使用的开源自托管内存中向量数据库，专为与LLM一起使用嵌入式设计。

在本节中，我们将：
- 实例化Chroma客户端
- 为每个嵌入类创建集合
- 查询每个集合


### 实例化Chroma客户端

创建Chroma客户端。默认情况下，Chroma是临时的，运行在内存中。
然而，您可以轻松地设置一个持久化配置，将数据写入磁盘。


In [5]:
chroma_client = chromadb.EphemeralClient() # 相当于 chromadb.Client()，临时性的。
# 取消注释以启用持久客户端
# chroma_client = chromadb.PersistentClient()


### 创建集合

Chroma集合允许您存储和过滤具有任意元数据的数据，从而可以轻松查询嵌入数据的子集。

Chroma已经集成了OpenAI的嵌入函数。最佳方法是在构建集合时使用它们，如下所示。
或者，您可以“自己带嵌入”。更多信息可以在[这里](https://docs.trychroma.com/embeddings)找到。


In [6]:
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

# 确保你的OpenAI API密钥已正确设置为环境变量。
# 注意：如果您在本地运行此笔记本，您需要重新加载终端和笔记本，以使环境变量生效。

# 注意：或者，您也可以像这样设置一个临时的环境变量：
# os.environ["OPENAI_API_KEY"] = 'sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

if os.getenv("OPENAI_API_KEY") is not None:
    openai.api_key = os.getenv("OPENAI_API_KEY")
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")


embedding_function = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'), model_name=EMBEDDING_MODEL)

wikipedia_content_collection = chroma_client.create_collection(name='wikipedia_content', embedding_function=embedding_function)
wikipedia_title_collection = chroma_client.create_collection(name='wikipedia_titles', embedding_function=embedding_function)


OPENAI_API_KEY is ready


### 填充集合

Chroma集合允许您填充和过滤任何您喜欢的元数据。Chroma还可以将文本存储在向量旁边，并在更方便时在单个`query`调用中返回所有内容。

对于这种用例，我们将只存储嵌入和ID，并使用它们来索引原始数据框。


In [15]:
# 添加内容向量
wikipedia_content_collection.add(
    ids=article_df.vector_id.tolist(),
    embeddings=article_df.content_vector.tolist(),
)

# 添加标题向量
wikipedia_title_collection.add(
    ids=article_df.vector_id.tolist(),
    embeddings=article_df.title_vector.tolist(),
)


### 搜索集合

如果设置了嵌入函数，Chroma会为您处理嵌入查询，就像这个例子中一样。


In [16]:
def query_collection(collection, query, max_results, dataframe):
    results = collection.query(query_texts=query, n_results=max_results, include=['distances']) 
    df = pd.DataFrame({
                'id':results['ids'][0], 
                'score':results['distances'][0],
                'title': dataframe[dataframe.vector_id.isin(results['ids'][0])]['title'],
                'content': dataframe[dataframe.vector_id.isin(results['ids'][0])]['text'],
                })
    
    return df


In [17]:
title_query_result = query_collection(
    collection=wikipedia_title_collection,
    query="modern art in Europe",
    max_results=10,
    dataframe=article_df
)
title_query_result.head()


,id,score,title,content
2,23266,0.249646,Art,Art is a creative activity that expresses imag...
11777,15436,0.271688,Hellenistic art,The art of the Hellenistic time (from 400 B.C....
12178,23265,0.279306,Byzantine art,Byzantine art is a form of Christian Greek art...
13215,11777,0.294415,Art film,Art films are a type of movie that is very dif...
15436,22108,0.305937,Renaissance art,Many of the most famous and best-loved works o...


In [18]:
content_query_result = query_collection(
    collection=wikipedia_content_collection,
    query="Famous battles in Scottish history",
    max_results=10,
    dataframe=article_df
)
content_query_result.head()


,id,score,title,content
2923,13135,0.261328,1651,\n\nEvents \n January 1 – Charles II crowned K...
3694,13571,0.277058,Stirling,Stirling () is a city in the middle of Scotlan...
6248,2923,0.294823,841,\n\nEvents \n June 25: Battle of Fontenay – Lo...
6297,13568,0.300756,1746,\n\nEvents \n January 8 – Bonnie Prince Charli...
11702,11708,0.307572,William Wallace,William Wallace was a Scottish knight who foug...


现在您已经运行了基本的嵌入搜索，您可以跳转到Chroma文档了解如何向查询添加过滤器、更新/删除集合中的数据以及部署Chroma。
